###### Sentiment Analysis

In [88]:
import tweepy           # Accessing Twitter's API
import pandas as pd     # To manipulate dataset
import numpy as np      # Used for statistical computation

In [89]:
# For plotting and visualization
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

In [90]:
consumer_key = 'KUMGIFXqLSLGLsr96JGWhVdwT'
consumer_secret = 'J5Igms3vZqq4JwJsTzw3qDUXDxDYD51r3g4l9zkiuZHVlRHs6t'
access_token = '275679909-vT2blBl9d0ldtiJJZR2EvvWbcWS0Yudi4jBOtCM6'
access_secret = '8DdHvBx1S7VHrZNbHvLRgKwvpJYq0JFwWKmZSDAz3nevL'

In [91]:
def twitter_setup():

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

extractor = twitter_setup()

In [92]:
# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="zomato", count=198)
print("Total tweets extracted: {}.\n".format(len(tweets)))

Total tweets extracted: 190.



In [93]:
# We print the most recent 5 tweets:
print("5 recent tweets:\n")
for tweet in tweets[:5]:
    print(tweet.text)

5 recent tweets:

taking over 150,000 steps to end the pandemic💉 https://t.co/uC1Sxd3icr
one indian dish that you wish had no calories?
@it_is_not_me_47 https://t.co/cvEeBiErPF
yes buy the dips saath mein fries mat bhoolna
the very first restaurant that you'd visit once things are back to normal?


In [94]:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
data.head(10)

,Tweets
0,"taking over 150,000 steps to end the pandemic💉..."
1,one indian dish that you wish had no calories?
2,@it_is_not_me_47 https://t.co/cvEeBiErPF
3,yes buy the dips saath mein fries mat bhoolna
4,the very first restaurant that you'd visit onc...
5,think we watched the wrong aspirants https://t...
6,@KFC_India @swiggy_in the credit goes to deliv...
7,just a tweet praying for the well-being of str...
8,the one we can't compete with\n#happymothersda...
9,we understand. we are just as tired of the sit...


In [95]:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

In [96]:
fav_max = np.max(data['Likes'])
rt_max  = np.max(data['RTs'])

In [97]:
fav = data[data.Likes == fav_max].index[0]
rt  = data[data.RTs == rt_max].index[0]

Max FAVs:

In [98]:
print("Most Liked Tweet: \n{}".format(data['Tweets'][fav]))

Most Liked Tweet: 
deliveries in Indiranagar miiight be late today due to an angry gunda on the road


In [99]:
print("Total Number of Tweets: {}".format(fav_max))

Total Number of Tweets: 27684


In [100]:
print("{} characters.\n".format(data['len'][fav]))

81 characters.



Max RTs:

In [101]:
print("Most re-tweeted Tweet: \n{}".format(data['Tweets'][rt]))

Most re-tweeted Tweet: 
RT @ZomatoIN: Food doesn’t have a religion. It is a religion. https://t.co/H8P5FlAw6y


In [102]:
print("Total Number of re-tweets: {}".format(rt_max))

Total Number of re-tweets: 25961


In [103]:
print("{} characters.\n".format(data['len'][rt]))

85 characters.



In [104]:
from textblob import TextBlob
import re

In [105]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [106]:
def analize_sentiment(tweet):

    sentiment_analysis = TextBlob(clean_tweet(tweet))
    if sentiment_analysis.sentiment.polarity > 0:
        return 1
    elif sentiment_analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [107]:
# We create a column with the result of the analysis:
data['SA'] = np.array([ analize_sentiment(tweet) for tweet in data['Tweets'] ])

In [108]:
# We display the updated dataframe with the new column:
display(data.head(10))

,Tweets,len,ID,Date,Source,Likes,RTs,SA
0,"taking over 150,000 steps to end the pandemic💉...",70,1395706841191960576,2021-05-21 11:43:36,Twitter Web App,217,15,0
1,one indian dish that you wish had no calories?,46,1395655056612077571,2021-05-21 08:17:50,Twitter Web App,1500,18,0
2,@it_is_not_me_47 https://t.co/cvEeBiErPF,40,1395024795910160386,2021-05-19 14:33:24,Twitter Web App,299,12,0
3,yes buy the dips saath mein fries mat bhoolna,45,1395023066489569280,2021-05-19 14:26:31,Twitter Web App,3391,242,0
4,the very first restaurant that you'd visit onc...,74,1394533619344068609,2021-05-18 06:01:38,Twitter Web App,689,17,1
5,think we watched the wrong aspirants https://t...,60,1394323965494038537,2021-05-17 16:08:33,Twitter Web App,1020,80,-1
6,@KFC_India @swiggy_in the credit goes to deliv...,60,1392735548071194626,2021-05-13 06:56:45,Twitter Web App,11,0,0
7,just a tweet praying for the well-being of str...,75,1392051403590230024,2021-05-11 09:38:12,Twitter Web App,1524,113,0
8,the one we can't compete with\n#happymothersda...,74,1391249696929193986,2021-05-09 04:32:30,Twitter Web App,1082,115,0
9,we understand. we are just as tired of the sit...,111,1391033071437180936,2021-05-08 14:11:43,Twitter Web App,1385,83,1


In [109]:
positive_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] > 0]
neutral_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 0]
negative_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] < 0]

In [110]:
print("Percentage of positive tweets: {}%".format(len(positive_tweets)*100/len(data['Tweets'])))

Percentage of positive tweets: 34.73684210526316%


In [111]:
print("Percentage of neutral tweets: {}%".format(len(neutral_tweets)*100/len(data['Tweets'])))

Percentage of neutral tweets: 55.78947368421053%


In [112]:
print("Percentage of negative tweets: {}%".format(len(negative_tweets)*100/len(data['Tweets'])))

Percentage of negative tweets: 9.473684210526315%
